# Pre-Processing Pipeline for Character Trajectories Dataset  

## Overview

This notebook contains the code for importing the raw CharacterTrajectories dataset and preparing it for use with a torchcde, LSTM, and RNN implementation.

In [1]:
import os
from sktime.datasets import load_from_tsfile_to_dataframe
import numpy as np
import torch

# Load the data
train_X, train_y = load_from_tsfile_to_dataframe('../../../data/raw/char_traj/CharacterTrajectories_TRAIN.ts')
test_X, test_y = load_from_tsfile_to_dataframe('../../../data/raw/char_traj/CharacterTrajectories_TEST.ts')

# Convert to NumPy arrays
train_X = train_X.to_numpy()
test_X = test_X.to_numpy()

# Concatenate the data to form a single dataset
X = np.concatenate((train_X, test_X), axis=0)   # (batch, channel)
y = np.concatenate((train_y, test_y), axis=0)   # (batch,)

# Change the labels to start from 0 and be integers
classes = np.unique(y)
y = np.array([np.where(classes == yi)[0][0] for yi in y])

# Make y a tensor
y = torch.tensor(y)

In [2]:
def _pad(channel, maxlen):
    channel = torch.tensor(channel)
    out = torch.full((maxlen,), 0, dtype=channel.dtype)
    out[:channel.size(0)] = channel
    return out

# Pad all data to same size with zeros. To be corrected for torchcde
lengths = torch.tensor([len(Xi[0]) for Xi in X])
maxlen = lengths.max()

X = torch.stack([torch.stack([_pad(channel, maxlen) for channel in batch], dim=0) for batch in X], dim=0)
X = X.transpose(-1, -2)

In [3]:
# Add time as a channel
times = [np.linspace(0, lengths[i] - 1, lengths[i]) for i in range(len(X))]
for i, time in enumerate(times):
    padding = np.zeros(maxlen - len(time))
    times[i] = np.concatenate((time, padding))

times = torch.tensor(times)

X = torch.cat((times.unsqueeze(-1), X), dim=-1)

/tmp/ipykernel_1054402/3901171844.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  times = torch.tensor(times)


In [6]:
def create_labels(X, lengths):
    '''
    Create labels for the data. The data is the first
    1/2 of each sequence and the labels are the last 1/2.
    Sequences are padded with zeros to be the same length.

    Parameters
    ----------
    X : torch.Tensor
        The data of shape (batch, seq_length, channels).
    lengths : torch.Tensor
        The lengths of the sequences.

    Returns
    -------
    data : torch.Tensor
        The data of shape (batch, seq_length, channels).
    labels : torch.Tensor
        The labels of shape (batch, seq_length, channels).
    '''
    data = torch.zeros(X.size(0), X.size(1) // 2, X.size(2))
    labels = torch.zeros(X.size(0), X.size(1) // 2 + 1, X.size(2))

    for i in range(X.size(0)):
        # Pad datapoint with zeros
        datapoint = X[i, :lengths[i] // 2]
        padding = torch.zeros(X.size(1) // 2 - len(datapoint), X.size(2))
        datapoint = torch.cat((datapoint, padding))

        # Pad label with zeros
        label = X[i, lengths[i] // 2:lengths[i]]
        padding = torch.zeros(X.size(1) // 2 - len(label) + 1, X.size(2))
        label = torch.cat((label, padding))
        
        data[i] = datapoint
        labels[i] = label

    return data, labels

data, labels = create_labels(X, lengths)

In [7]:
# Split the data into train and test
np.random.seed(0)

indices = np.arange(len(X))
np.random.shuffle(indices)
X_shuffled = X[indices]
y_shuffled = y[indices]

test_size = 0.2
num_test = int(test_size * len(X))

X_train, X_test = X_shuffled[:-num_test], X_shuffled[-num_test:]
y_train, y_test = y_shuffled[:-num_test], y_shuffled[-num_test:]

# Save the data
path = '../../data/processed/CharacterTrajectories/regression'
os.makedirs(path, exist_ok=True)
torch.save(X_train, os.path.join(path, 'X_train.pt'))
torch.save(y_train, os.path.join(path, 'y_train.pt'))
torch.save(X_test, os.path.join(path, 'X_test.pt'))
torch.save(y_test, os.path.join(path, 'y_test.pt'))